# Análisis del Mercado Eléctrico Italiano

## Contexto
Este análisis se basa en un conjunto de datos sobre el mercado eléctrico italiano, disponible en Kaggle. El dataset cubre el periodo de 2012 a 2022 y proporciona información sobre la demanda de electricidad, las fuentes de producción de energía y los precios de la electricidad en Italia. Durante este periodo, Italia ha enfrentado una transición energética debido a cambios en la oferta y la demanda de energía, impulsados por factores como las interrupciones en el suministro de gas, la mayor inversión en energías renovables y las tensiones geopolíticas. Estos cambios son cruciales para comprender las variaciones en la demanda de electricidad.

El objetivo de este análisis es explorar y entender cómo ha evolucionado la demanda eléctrica a lo largo de los años, comparando las distintas regiones del país y realizando predicciones sobre la futura demanda. Este análisis es clave para la optimización de la producción y distribución de energía.

## Archivos del Dataset
El conjunto de datos está compuesto por varios archivos CSV que contienen información específica sobre la producción, la demanda y los precios de la electricidad en Italia. A continuación se describen los archivos y sus características:

1. **`it_electricity_demand_adj.csv`** (7.94 MB)
   - **Descripción**: Contiene datos ajustados sobre la demanda eléctrica horaria en Italia durante el período cubierto (2012-2022).
   - **Características**: Detalla el consumo de electricidad por hora en diferentes regiones de Italia (por ejemplo, Norte, Sur, Centro-Norte, etc.).

2. **`it_electricity_prices_adj.csv`** (11.97 MB)
   - **Descripción**: Contiene información sobre los precios horarios de la electricidad en Italia, ajustados para reflejar las tendencias económicas.
   - **Características**: Información sobre el precio de la electricidad por megavatio hora (€/MWh), lo que permite observar las fluctuaciones de los precios a lo largo del tiempo.

3. **`electricity-prod-source-stacked.csv`** (377.27 KB)
   - **Descripción**: Proporciona información sobre la producción de electricidad en Italia, desglosada por fuente de energía (renovables, fósiles, etc.).
   - **Características**: Contiene la proporción de electricidad generada por cada tipo de fuente de energía, lo que es esencial para comprender cómo varía la oferta de electricidad según las fuentes de energía.

4. **`NYMEX_DL_TTF1_1D.csv`** (73.38 KB)
   - **Descripción**: Contiene datos sobre los futuros continuos del gas natural holandés (Dutch Natural Gas Continuous Future), que es relevante para entender los precios de la energía en Europa.
   - **Características**: Datos de precios futuros del gas natural, lo cual puede influir en el precio de la electricidad.
   

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Cargar los datos
demanda_electrica_horaria_it_df = pd.read_csv('it_electricity_demand_adj.csv', index_col=None)
prices_electricidad_ph_df = pd.read_csv('it_electricity_prices_adj.csv', index_col=None)
prod_electricidad__df = pd.read_csv('electricity-prod-source-stacked.csv', index_col=None)
gas_precios_futuros_df = pd.read_csv('NYMEX_DL_TTF1 1D.csv')




### LIMPIAR Y ANALIZAR EL PRIMER DATASET

In [97]:
print('Info dataset demanda electrica horaria en Italia desde 2012 a 2022:')
print(demanda_electrica_horaria_it_df.info())

Info dataset demanda electrica horaria en Italia desde 2012 a 2022:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93504 entries, 0 to 93503
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  93504 non-null  object 
 1   Italia      93504 non-null  float64
 2   CNOR        93504 non-null  float64
 3   CSUD        93504 non-null  float64
 4   NORD        93504 non-null  float64
 5   SARD        93504 non-null  float64
 6   SICI        93504 non-null  float64
 7   SUD         93504 non-null  float64
dtypes: float64(7), object(1)
memory usage: 5.7+ MB
None


In [98]:
# Convertir la columna 'Unnamed: 0' a formato de fecha y hora
demanda_electrica_horaria_it_df['Unnamed: 0'] = pd.to_datetime(demanda_electrica_horaria_it_df['Unnamed: 0'])

# renombrar la columna "Unnamed" a "datetime"
demanda_electrica_horaria_it_df = demanda_electrica_horaria_it_df.rename(columns={'Unnamed: 0': 'DATE', 'Italia': 'ITALIA'})

# Verificar la conversión y la estructura de los datos
print(demanda_electrica_horaria_it_df.info())
print('--------------------------------------------------------------------------------------------------------------------------')
print(demanda_electrica_horaria_it_df.head())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93504 entries, 0 to 93503
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    93504 non-null  datetime64[ns]
 1   ITALIA  93504 non-null  float64       
 2   CNOR    93504 non-null  float64       
 3   CSUD    93504 non-null  float64       
 4   NORD    93504 non-null  float64       
 5   SARD    93504 non-null  float64       
 6   SICI    93504 non-null  float64       
 7   SUD     93504 non-null  float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 5.7 MB
None
--------------------------------------------------------------------------------------------------------------------------
                 DATE     ITALIA      CNOR      CSUD       NORD      SARD  \
0 2012-01-01 01:00:00  24659.161  2662.703  4534.718  11693.113  1460.687   
1 2012-01-01 02:00:00  23387.590  2539.670  4240.313  11113.677  1437.934   
2 2012-01-01 03:00:00  22090.760  

In [99]:
# Comprobar si hay valores nulos en demanda_electrica_horaria_it_df
print('VALORES NULOS:')
print(demanda_electrica_horaria_it_df.isnull().sum())
print('-----------------------------------------------------------------------------------')
print(demanda_electrica_horaria_it_df.describe())

VALORES NULOS:
DATE      0
ITALIA    0
CNOR      0
CSUD      0
NORD      0
SARD      0
SICI      0
SUD       0
dtype: int64
-----------------------------------------------------------------------------------
                      DATE        ITALIA          CNOR          CSUD  \
count                93504  93504.000000  93504.000000  93504.000000   
mean   2017-05-02 00:30:00  32519.967098   3287.953532   5247.286205   
min    2012-01-01 01:00:00  16272.977000   1550.070000   2580.443000   
25%    2014-09-01 00:45:00  26502.196750   2643.802000   4318.080750   
50%    2017-05-02 00:30:00  32016.382500   3224.852000   5258.185500   
75%    2020-01-01 00:15:00  38450.967750   3913.896750   6099.085000   
max    2022-09-01 00:00:00  50393.780000   6007.450000   8899.763000   
std                    NaN   6818.933965    791.864286   1128.337246   

               NORD          SARD          SICI           SUD  
count  93504.000000  93504.000000  93504.000000  93504.000000  
mean   18044.11

In [100]:
# funciones para definir estación y momento del día
def asignar_estacion_en_ingles(mes):
    if mes in [12, 1, 2]:
        return 'Winter'
    elif mes in [3, 4, 5]:
        return 'Spring'
    elif mes in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'
    
def asignar_momento_dia_en_ingles(hora):
    if hora >= 0 and hora < 6:
        return 'Night'
    elif hora >= 6 and hora < 12:
        return 'Morning'
    elif hora >= 12 and hora < 18:
        return 'Afternoon'    
    else:
        return 'Evening'
    
# Añadir las columnas 'estaciòn', 'momento del dia', 'año' y 'mes' al dataframe.
demanda_electrica_horaria_it_df['YEAR'] = demanda_electrica_horaria_it_df['DATE'].dt.year
demanda_electrica_horaria_it_df['MONTH'] = demanda_electrica_horaria_it_df['DATE'].dt.month_name()
demanda_electrica_horaria_it_df['SEASON'] = demanda_electrica_horaria_it_df['DATE'].dt.month.apply(asignar_estacion_en_ingles)
demanda_electrica_horaria_it_df['TIME_OF_DAY'] = demanda_electrica_horaria_it_df['DATE'].dt.hour.apply(asignar_momento_dia_en_ingles)

print(demanda_electrica_horaria_it_df)


                     DATE     ITALIA      CNOR      CSUD       NORD      SARD  \
0     2012-01-01 01:00:00  24659.161  2662.703  4534.718  11693.113  1460.687   
1     2012-01-01 02:00:00  23387.590  2539.670  4240.313  11113.677  1437.934   
2     2012-01-01 03:00:00  22090.760  2403.112  3927.174  10558.286  1410.237   
3     2012-01-01 04:00:00  21009.116  2267.737  3692.188  10126.168  1379.016   
4     2012-01-01 05:00:00  20350.988  2186.529  3569.043   9851.236  1340.877   
...                   ...        ...       ...       ...        ...       ...   
93499 2022-08-31 20:00:00  39234.945  3450.666  6835.219  21600.813  1233.899   
93500 2022-08-31 21:00:00  39005.320  3417.592  6790.829  21280.509  1260.336   
93501 2022-08-31 22:00:00  38956.535  3358.284  6855.084  20964.329  1305.351   
93502 2022-08-31 23:00:00  37077.835  3172.588  6478.296  20155.819  1250.641   
93503 2022-09-01 00:00:00  34155.930  2868.968  5940.663  18603.728  1162.996   

           SICI       SUD  

In [101]:
# Eliminar las filas correspondientes al año 2022 que no esta completo, los datos llegan hasta el 1 de Septiembre 2022
demanda_electrica_horaria_it_df_sin_2022 = demanda_electrica_horaria_it_df[demanda_electrica_horaria_it_df['YEAR'] < 2022]

# Verificar que las filas de 2022 han sido eliminadas
print(demanda_electrica_horaria_it_df_sin_2022.tail())

                     DATE     ITALIA      CNOR      CSUD       NORD      SARD  \
87666 2021-12-31 19:00:00  33594.570  3056.068  6486.964  17214.477  1158.396   
87667 2021-12-31 20:00:00  33059.312  3009.514  6497.047  16777.262  1160.444   
87668 2021-12-31 21:00:00  30955.712  2766.744  6079.317  15630.957  1131.324   
87669 2021-12-31 22:00:00  28636.193  2542.831  5616.364  14531.612  1056.545   
87670 2021-12-31 23:00:00  26498.170  2319.290  5246.229  13428.359   989.251   

           SICI       SUD  YEAR     MONTH  SEASON TIME_OF_DAY  
87666  2462.186  2328.333  2021  December  Winter     Evening  
87667  2450.344  2281.282  2021  December  Winter     Evening  
87668  2314.756  2212.135  2021  December  Winter     Evening  
87669  2131.281  2013.634  2021  December  Winter     Evening  
87670  1977.628  1851.318  2021  December  Winter     Evening  


In [102]:
# AGRUPAR POR AÑO Y VER LA DEMANDA TOTAL DE CDA ZONA POR CADA AÑO
demand_por_zona_por_anio = demanda_electrica_horaria_it_df_sin_2022.groupby(['YEAR']).agg({
    'ITALIA': 'sum',  # Sumar la columna 'Italia'
    'CNOR': 'sum',    # Sumar la columna 'CNOR'
    'CSUD': 'sum',    # Sumar la columna 'CSUD'
    'NORD': 'sum',    # Sumar la columna 'NORD'
    'SARD': 'sum',    # Sumar la columna 'SARD'
    'SICI': 'sum',    # Sumar la columna 'SICI'
    'SUD': 'sum'
}).reset_index()

print(demand_por_zona_por_anio)

   YEAR        ITALIA          CNOR          CSUD          NORD          SARD  \
0  2012  2.945348e+08  3.157968e+07  4.767508e+07  1.577065e+08  1.263108e+07   
1  2013  2.852032e+08  2.851774e+07  4.433649e+07  1.564274e+08  1.089503e+07   
2  2014  2.784616e+08  2.586894e+07  4.064183e+07  1.565127e+08  1.141631e+07   
3  2015  2.826228e+08  2.811454e+07  4.497811e+07  1.558118e+08  8.868341e+06   
4  2016  2.822920e+08  2.985886e+07  4.615561e+07  1.552723e+08  8.748406e+06   
5  2017  2.860419e+08  3.124860e+07  4.657017e+07  1.591522e+08  8.813259e+06   
6  2018  2.917606e+08  3.108157e+07  4.594104e+07  1.644496e+08  8.974319e+06   
7  2019  2.889769e+08  3.095200e+07  4.574050e+07  1.620192e+08  8.908264e+06   
8  2020  2.715093e+08  2.890833e+07  4.384289e+07  1.507080e+08  8.405581e+06   
9  2021  2.860881e+08  2.457109e+07  5.061676e+07  1.609361e+08  8.857971e+06   

           SICI           SUD  
0  1.991133e+07  2.503108e+07  
1  1.932450e+07  2.570209e+07  
2  1.804945e

### LIMPIAR Y ANALIZAR EL SEGUNDO DATASET

In [103]:

print('Info dataset precios horarios electricidad en Italia:')
print(prices_electricidad_ph_df.info())


Info dataset precios horarios electricidad en Italia:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93504 entries, 0 to 93503
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  93504 non-null  object 
 1   PUN         93504 non-null  float64
 2   NAT         93504 non-null  float64
 3   CNOR        93504 non-null  float64
 4   CSUD        93504 non-null  float64
 5   NORD        93504 non-null  float64
 6   SARD        93504 non-null  float64
 7   SICI        93504 non-null  float64
 8   SUD         93504 non-null  float64
 9   AUST        93504 non-null  float64
 10  COAC        93504 non-null  float64
 11  CORS        93504 non-null  float64
 12  FRAN        93504 non-null  float64
 13  GREC        93504 non-null  float64
 14  SLOV        93504 non-null  float64
 15  SVIZ        93504 non-null  float64
 16  BSP         93504 non-null  float64
 17  CALA        744 non-null    float64
 18  COUP        

In [104]:
# VAMOS A EXTRAER SOLO LAS COLUMNAS QUE NOS INTERESAN Y LAS VAMOS A RENOMBRAR

columnas_a_analizar = ['Unnamed: 0', 'PUN', 'NAT', 'CNOR', 'CSUD', 'NORD', 'SARD', 'SICI', 'SUD']
nuevos_nombres = {
    'Unnamed: 0': 'DATE',
    'PUN': 'UNIQUE_NATIONAL_PRICE',
    'NAT': 'UNCONSTRAINED_PRICE',
    'CNOR': 'NORD_CENTER_PRICE',
    'CSUD': 'SUD_CENTER_PRICE',
    'NORD': 'NORD_PRICE',
    'SARD': 'SARDINA_PRICE',
    'SICI': 'SICILY_PRICE',
    'SUD': 'SUD_PRICE'
}

# actualizar el dataframe con las columnas seleccionadas
prices_electricidad_ph_df = prices_electricidad_ph_df[columnas_a_analizar].rename(columns=nuevos_nombres)

# Convertir la columna DATE a formato datetime
prices_electricidad_ph_df['DATE'] = pd.to_datetime(prices_electricidad_ph_df['DATE'])

print(prices_electricidad_ph_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93504 entries, 0 to 93503
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   DATE                   93504 non-null  datetime64[ns]
 1   UNIQUE_NATIONAL_PRICE  93504 non-null  float64       
 2   UNCONSTRAINED_PRICE    93504 non-null  float64       
 3   NORD_CENTER_PRICE      93504 non-null  float64       
 4   SUD_CENTER_PRICE       93504 non-null  float64       
 5   NORD_PRICE             93504 non-null  float64       
 6   SARDINA_PRICE          93504 non-null  float64       
 7   SICILY_PRICE           93504 non-null  float64       
 8   SUD_PRICE              93504 non-null  float64       
dtypes: datetime64[ns](1), float64(8)
memory usage: 6.4 MB
None


In [105]:
# Comprobar si hay valores nulos en prices_electricidad_ph_df
print('VALORES NULOS:')
print(prices_electricidad_ph_df.isnull().sum())
print('-----------------------------------------------------------------------------------')
print(prices_electricidad_ph_df.describe())

VALORES NULOS:
DATE                     0
UNIQUE_NATIONAL_PRICE    0
UNCONSTRAINED_PRICE      0
NORD_CENTER_PRICE        0
SUD_CENTER_PRICE         0
NORD_PRICE               0
SARDINA_PRICE            0
SICILY_PRICE             0
SUD_PRICE                0
dtype: int64
-----------------------------------------------------------------------------------
                      DATE  UNIQUE_NATIONAL_PRICE  UNCONSTRAINED_PRICE  \
count                93504           93504.000000         93504.000000   
mean   2017-05-02 00:30:00              77.307662            76.245255   
min    2012-01-01 01:00:00               0.000000             0.000000   
25%    2014-09-01 00:45:00              43.785658            43.000000   
50%    2017-05-02 00:30:00              56.000000            55.000000   
75%    2020-01-01 00:15:00              73.214997            71.430000   
max    2022-09-01 00:00:00             870.000000          3000.000000   
std                    NaN              78.206100    

In [106]:
# Añadir las columnas 'estaciòn', 'momento del dia', 'año' y 'mes' al dataframe.
prices_electricidad_ph_df['YEAR'] = prices_electricidad_ph_df['DATE'].dt.year
prices_electricidad_ph_df['MONTH'] = prices_electricidad_ph_df['DATE'].dt.month_name()
prices_electricidad_ph_df['SEASON'] = prices_electricidad_ph_df['DATE'].dt.month.apply(asignar_estacion_en_ingles)
prices_electricidad_ph_df['TIME_OF_DAY'] = prices_electricidad_ph_df['DATE'].dt.hour.apply(asignar_momento_dia_en_ingles)

print(prices_electricidad_ph_df)

                     DATE  UNIQUE_NATIONAL_PRICE  UNCONSTRAINED_PRICE  \
0     2012-01-01 01:00:00              82.813341                76.29   
1     2012-01-01 02:00:00              75.767425                65.80   
2     2012-01-01 03:00:00              59.919853                60.00   
3     2012-01-01 04:00:00              59.920928                59.03   
4     2012-01-01 05:00:00              59.010000                59.00   
...                   ...                    ...                  ...   
93499 2022-08-31 20:00:00             672.870080               677.88   
93500 2022-08-31 21:00:00             743.531820               744.00   
93501 2022-08-31 22:00:00             759.990000               759.99   
93502 2022-08-31 23:00:00             706.072680               706.48   
93503 2022-09-01 00:00:00             639.300000               639.30   

       NORD_CENTER_PRICE  SUD_CENTER_PRICE  NORD_PRICE  SARDINA_PRICE  \
0               83.95000          83.95000    83.9

In [107]:
# VAMOS A QUITAR TODOS LOS DATOS DEL 2022 QUE NO ESTÀ COMPLETO,SOLO TENEMOS DATOS HASTA EL 1 DE SEPTIEMBRE 2022

prices_electricidad_ph_df = prices_electricidad_ph_df[prices_electricidad_ph_df['YEAR'] < 2022]

# VERIFICAMOS QUE EL 2022 HA SIDO ELIMINADO DEL DATAFRAME

print(prices_electricidad_ph_df.tail())

                     DATE  UNIQUE_NATIONAL_PRICE  UNCONSTRAINED_PRICE  \
87666 2021-12-31 19:00:00                 221.76               221.76   
87667 2021-12-31 20:00:00                 219.14               219.14   
87668 2021-12-31 21:00:00                 213.60               213.60   
87669 2021-12-31 22:00:00                 192.79               192.79   
87670 2021-12-31 23:00:00                 180.00               180.00   

       NORD_CENTER_PRICE  SUD_CENTER_PRICE  NORD_PRICE  SARDINA_PRICE  \
87666             221.76            221.76      221.76         221.76   
87667             219.14            219.14      219.14         219.14   
87668             213.60            213.60      213.60         213.60   
87669             192.79            192.79      192.79         192.79   
87670             180.00            180.00      180.00         180.00   

       SICILY_PRICE  SUD_PRICE  YEAR     MONTH  SEASON TIME_OF_DAY  
87666        221.76     221.76  2021  December  Winte

In [108]:
prices_electricidad_ph_df = prices_electricidad_ph_df.groupby(['YEAR']).agg({
    'UNIQUE_NATIONAL_PRICE': 'sum',  # Sumar la columna 'Italia'
    'NORD_CENTER_PRICE': 'sum',    # Sumar la columna 'CNOR'
    'SUD_CENTER_PRICE': 'sum',    # Sumar la columna 'CSUD'
    'NORD_PRICE': 'sum',    # Sumar la columna 'NORD'
    'SARDINA_PRICE': 'sum',    # Sumar la columna 'SARD'
    'SICILY_PRICE': 'sum',    # Sumar la columna 'SICI'
    'SUD_PRICE': 'sum',
    'UNCONSTRAINED_PRICE': 'sum'
}).reset_index()

print(prices_electricidad_ph_df)

   YEAR  UNIQUE_NATIONAL_PRICE  NORD_CENTER_PRICE  SUD_CENTER_PRICE  \
0  2012           6.629322e+05       6.488060e+05      6.425980e+05   
1  2013           5.517707e+05       5.348170e+05      5.191146e+05   
2  2014           4.562134e+05       4.343202e+05      4.280411e+05   
3  2015           4.582076e+05       4.505942e+05      4.454027e+05   
4  2016           3.757488e+05       3.737812e+05      3.653161e+05   
5  2017           4.726153e+05       4.736617e+05      4.520961e+05   
6  2018           5.370721e+05       5.349200e+05      5.338368e+05   
7  2019           4.583740e+05       4.575703e+05      4.579496e+05   
8  2020           3.418327e+05       3.399789e+05      3.484282e+05   
9  2021           1.098931e+06       1.098277e+06      1.097471e+06   

     NORD_PRICE  SARDINA_PRICE  SICILY_PRICE     SUD_PRICE  \
0  6.503885e+05   7.173337e+05  8.368290e+05  6.177844e+05   
1  5.394934e+05   5.389026e+05  8.059153e+05  5.012309e+05   
2  4.410887e+05   4.571279e+05  

### LIMPIAR Y ANALIZAR EL TERCER DATAFRAME

In [109]:
print('Info dataset producción energia electrica en Italia por fuente de energia:')
print(prod_electricidad__df.info())


Info dataset producción energia electrica en Italia por fuente de energia:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7995 entries, 0 to 7994
Data columns (total 11 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Entity                                                       7995 non-null   object 
 1   Code                                                         7038 non-null   object 
 2   Year                                                         7995 non-null   int64  
 3   Electricity from coal (TWh)                                  5747 non-null   float64
 4   Electricity from gas (TWh)                                   5748 non-null   float64
 5   Electricity from hydro (TWh)                                 7994 non-null   float64
 6   Electricity from other renewables including bioenergy (TWh)  7989 non-null   float64
 7   Ele

In [110]:
# en primer lugar vamos a extraer solo las filas donde el entity es igual a Italy
prod_electricidad__df = prod_electricidad__df[prod_electricidad__df['Entity'] == 'Italy']
print(prod_electricidad__df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 57 entries, 3556 to 3612
Data columns (total 11 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Entity                                                       57 non-null     object 
 1   Code                                                         57 non-null     object 
 2   Year                                                         57 non-null     int64  
 3   Electricity from coal (TWh)                                  37 non-null     float64
 4   Electricity from gas (TWh)                                   37 non-null     float64
 5   Electricity from hydro (TWh)                                 57 non-null     float64
 6   Electricity from other renewables including bioenergy (TWh)  57 non-null     float64
 7   Electricity from solar (TWh)                                 57 non-null     float

In [111]:
# Comprobar si hay valores nulos en demanda_electrica_horaria_it_df
print('VALORES NULOS:')
print(prod_electricidad__df.isnull().sum())



VALORES NULOS:
Entity                                                          0
Code                                                            0
Year                                                            0
Electricity from coal (TWh)                                    20
Electricity from gas (TWh)                                     20
Electricity from hydro (TWh)                                    0
Electricity from other renewables including bioenergy (TWh)     0
Electricity from solar (TWh)                                    0
Electricity from oil (TWh)                                     20
Electricity from wind (TWh)                                     0
Electricity from nuclear (TWh)                                  0
dtype: int64


In [112]:

# en este caso vamos a poner 0.0 como float en cambio de los nulos

prod_electricidad__df = prod_electricidad__df.fillna(float(0.0))
print(prod_electricidad__df.isnull().sum())


Entity                                                         0
Code                                                           0
Year                                                           0
Electricity from coal (TWh)                                    0
Electricity from gas (TWh)                                     0
Electricity from hydro (TWh)                                   0
Electricity from other renewables including bioenergy (TWh)    0
Electricity from solar (TWh)                                   0
Electricity from oil (TWh)                                     0
Electricity from wind (TWh)                                    0
Electricity from nuclear (TWh)                                 0
dtype: int64


In [113]:
# vamos a organizar el dataframe quitando las columnas que ya no intersan como del entity  y del code
columnas_a_selecionar = [
    'Year', 
    'Electricity from coal (TWh)', 
    'Electricity from gas (TWh)', 
    'Electricity from hydro (TWh)', 
    'Electricity from other renewables including bioenergy (TWh)', 
    'Electricity from solar (TWh)', 
    'Electricity from oil (TWh)', 
    'Electricity from wind (TWh)', 
    'Electricity from nuclear (TWh)'
]

prod_electricidad__df = prod_electricidad__df[columnas_a_selecionar]

# resetear el index
prod_electricidad__df = prod_electricidad__df.reset_index(drop=True)
print(prod_electricidad__df.head())



   Year  Electricity from coal (TWh)  Electricity from gas (TWh)  \
0  1985                       30.023                      24.712   
1  1986                       31.004                      26.871   
2  1987                       33.035                      31.732   
3  1988                       33.897                      32.420   
4  1989                       31.867                      34.407   

   Electricity from hydro (TWh)  \
0                        41.092   
1                        41.096   
2                        39.505   
3                        40.676   
4                        34.057   

   Electricity from other renewables including bioenergy (TWh)  \
0                                              3.212             
1                                              3.501             
2                                              3.693             
3                                              3.787             
4                                              3.1

In [114]:
# pasar los valores de TWh a MWh
columns_to_convert = [
    'Electricity from coal (TWh)', 
    'Electricity from gas (TWh)', 
    'Electricity from hydro (TWh)', 
    'Electricity from other renewables including bioenergy (TWh)', 
    'Electricity from solar (TWh)', 
    'Electricity from oil (TWh)', 
    'Electricity from wind (TWh)', 
    'Electricity from nuclear (TWh)'
]

for col in columns_to_convert:
    prod_electricidad__df[col] *= 1000000
    

# Renombrar columnas automáticamente
prod_electricidad__df = prod_electricidad__df.rename(columns={col: col.replace('TWh', 'MWh') for col in columns_to_convert})

print(prod_electricidad__df.head())


   Year  Electricity from coal (MWh)  Electricity from gas (MWh)  \
0  1985                   30023000.0                  24712000.0   
1  1986                   31004000.0                  26871000.0   
2  1987                   33035000.0                  31732000.0   
3  1988                   33897000.0                  32420000.0   
4  1989                   31867000.0                  34407000.0   

   Electricity from hydro (MWh)  \
0                    41092000.0   
1                    41096000.0   
2                    39505000.0   
3                    40676000.0   
4                    34057000.0   

   Electricity from other renewables including bioenergy (MWh)  \
0                                          3212000.0             
1                                          3501000.0             
2                                          3693000.0             
3                                          3787000.0             
4                                          3197000

### LIMPIAR Y ANALIZAR CUARTO DATAFRAME

In [115]:
print('Info dataset precios futuros del gas natural en Europa:')
print(gas_precios_futuros_df.info())

Info dataset precios futuros del gas natural en Europa:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       1224 non-null   object 
 1   open       1224 non-null   float64
 2   high       1224 non-null   float64
 3   low        1224 non-null   float64
 4   close      1224 non-null   float64
 5   Volume     1224 non-null   int64  
 6   Volume MA  1205 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 67.1+ KB
None


In [116]:
# VAMOS A PASAR LA COLUMNA TIME A DATETIEM Y LA RENOMBRAMOS A "DATE"

gas_precios_futuros_df['time'] = pd.to_datetime(gas_precios_futuros_df['time'], utc=True)
gas_precios_futuros_df = gas_precios_futuros_df.rename(columns={'time':'DATE'})
print(gas_precios_futuros_df.head())


                       DATE   open   high    low  close  Volume  Volume MA
0 2017-10-22 22:00:00+00:00  18.09  18.09  18.09  18.09       0        NaN
1 2017-10-23 22:00:00+00:00  17.96  17.96  17.96  17.96       0        NaN
2 2017-10-24 22:00:00+00:00  18.11  18.11  18.11  18.11       0        NaN
3 2017-10-25 22:00:00+00:00  18.07  18.07  18.07  18.07       0        NaN
4 2017-10-26 22:00:00+00:00  18.15  18.15  18.15  18.15       0        NaN


In [117]:
# ver los valores nulos
print('Valores nulos:')
print(gas_precios_futuros_df.isnull().sum())


Valores nulos:
DATE          0
open          0
high          0
low           0
close         0
Volume        0
Volume MA    19
dtype: int64


In [118]:
# pasar la columna Volume a float
gas_precios_futuros_df['Volume'] = gas_precios_futuros_df['Volume'].astype('float64')
print(gas_precios_futuros_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   DATE       1224 non-null   datetime64[ns, UTC]
 1   open       1224 non-null   float64            
 2   high       1224 non-null   float64            
 3   low        1224 non-null   float64            
 4   close      1224 non-null   float64            
 5   Volume     1224 non-null   float64            
 6   Volume MA  1205 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(6)
memory usage: 67.1 KB
None


In [119]:
# ordenar el dataframe por la fecha mas antigua hasta la mas recien
gas_precios_futuros_df= gas_precios_futuros_df.sort_values(by='DATE', ascending=True)
print(gas_precios_futuros_df.head(20))

                        DATE    open    high     low   close  Volume  \
0  2017-10-22 22:00:00+00:00  18.090  18.090  18.090  18.090     0.0   
1  2017-10-23 22:00:00+00:00  17.960  17.960  17.960  17.960     0.0   
2  2017-10-24 22:00:00+00:00  18.110  18.110  18.110  18.110     0.0   
3  2017-10-25 22:00:00+00:00  18.070  18.070  18.070  18.070     0.0   
4  2017-10-26 22:00:00+00:00  18.150  18.150  18.150  18.150     0.0   
5  2017-10-29 22:00:00+00:00  18.110  18.110  18.110  18.110     0.0   
6  2017-10-30 22:00:00+00:00  18.285  18.285  18.285  18.285     0.0   
7  2017-10-31 22:00:00+00:00  18.550  18.550  18.550  18.550     0.0   
8  2017-11-01 22:00:00+00:00  18.580  18.580  18.580  18.580     0.0   
9  2017-11-02 22:00:00+00:00  18.310  18.310  18.310  18.310     0.0   
10 2017-11-05 23:00:00+00:00  19.010  19.010  19.010  19.010     0.0   
11 2017-11-06 23:00:00+00:00  19.325  19.325  19.325  19.325     0.0   
12 2017-11-07 23:00:00+00:00  19.380  19.380  19.380  19.380    

In [120]:
# ver donde estàn los nulos, osea si son consecutivos en el tiempo o no
print(gas_precios_futuros_df.isnull())

       DATE   open   high    low  close  Volume  Volume MA
0     False  False  False  False  False   False       True
1     False  False  False  False  False   False       True
2     False  False  False  False  False   False       True
3     False  False  False  False  False   False       True
4     False  False  False  False  False   False       True
...     ...    ...    ...    ...    ...     ...        ...
1219  False  False  False  False  False   False      False
1220  False  False  False  False  False   False      False
1221  False  False  False  False  False   False      False
1222  False  False  False  False  False   False      False
1223  False  False  False  False  False   False      False

[1224 rows x 7 columns]
